In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import os.path
MODEL_FILE ="flowers.hd5"

In [4]:
def create_model(num_hidden, num_classes):
# We get the base model using InceptionV3 and the imagenet
# weights that was trained on tens of thousands of images.
 base_model = InceptionV3(include_top = False, weights = 'imagenet')
# Get the output layer, then does an average pooling of this
# output, and feeds it to a final Dense layer that we
# will train
 x = base_model.output
 x = GlobalAveragePooling2D()(x)
 x = Dense(num_hidden, activation='relu')(x)
 predictions = Dense(num_classes, activation='softmax')(x)
# Set base model layers to be non trainable so we focus
# our training only in the Dense layer. This lets us
# adapt much faster and doesn't corrupt the weights that
# were already trained on imagenet.
 for layer in base_model.layers:
   layer.trainable = False
# Create a Functional Model (as opposed to the usual
# Sequential Model that we create
 model = Model(inputs=base_model.input, outputs = predictions)
 return model

In [5]:
# Loads an existing model file, then sets only the last
# 3 layers (which we added) to trainable.
def load_existing(model_file) :
# Load the model
 model = load_model(model_file)
# Set only last 3 layers as trainable
 numlayers = len(model.layers)
 for layer in model.layers[:numlayers-3]:
  layer.trainable = False
# Set remaining layers to be trainable.
 for layer in model.layers[numlayers-3:]:
  layer .trainable = True
 return model

In [6]:
def train(model_file,train_path,validation_path,num_hidden=200,num_classes=5,steps=32,num_epochs=20):
# If an existing model exists, we load it. Otherwise we create a
# new model from scratch
 if os.path.exists(model_file):
  print("\n*** Existing model found at %s. Loading.***\n\n" % model_file)
  model = load_existing(model_file)
 else:
  print("\n*** Creating new model ***\n\n")
  model = create_model(num_hidden, num_classes)
# Since we have multiple categories and a softmax output
 model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Create a checkpoint to save the model after every epo
 checkpoint = ModelCheckpoint(model_file)
# Now we create a generator. This will take our image data, rescale it.
# shear it, zoom in and out to create additional images for training.
 train_datagen = ImageDataGenerator(\
 rescale=1./255,\
 shear_range=0.2,\
 zoom_range=0.2,horizontal_flip = True)
# Image generator for test data
 test_datagen = ImageDataGenerator(rescale=1./255)
 train_generator = train_datagen.flow_from_directory(\
 train_path,\
 target_size=(249,249),
 batch_size=5,
 class_mode ="categorical")
# We do the same for the validation set .
 validation_generator  = test_datagen.flow_from_directory(\
 validation_path,\
 target_size=(249,249),
 batch_size=5,
 class_mode ="categorical")

# Finally we train the neural network
 model.fit(\
 train_generator,\
 steps_per_epoch = steps,\
 epochs =num_epochs,\
 callbacks = [checkpoint],\
 validation_data= validation_generator,\
 validation_steps = 50)
# Train last two layers
# Now we twek the training by freezing almost all the layers and
# just train the topmost layer
 for layer  in model.layers[:249]:
   layer.trainable = False
 for layer in model.layers[249:]:
   layer .trainable = True

 model.compile(optimizer=SGD(learning_rate=0.00001, momentum=0.9), loss='categorical_crossentropy')
 model.fit(\
 train_generator,\
 steps_per_epoch=steps,\
 epochs = num_epochs,\
 callbacks = [checkpoint],\
 validation_data = validation_generator,\
 validation_steps = 50)

In [7]:
def main() :
  train(MODEL_FILE,train_path="/content/gdrive/My Drive/data/flower_photos",
        validation_path="/content/gdrive/My Drive/data/flower_photos",steps=120,num_epochs=10)
if __name__ == "__main__":
  main()


*** Creating new model ***


87910968/87910968 [==============================] - 0s 0us/step
Found 860 images belonging to 5 classes.
Found 860 images belonging to 5 classes.
Epoch 1/10
120/120 [==============================] - ETA: 0s - loss: 0.3171

120/120 [==============================] - 84s 505ms/step - loss: 0.3171 - val_loss: 0.1201
Epoch 2/10
120/120 [==============================] - ETA: 0s - loss: 0.1057

120/120 [==============================] - 47s 395ms/step - loss: 0.1057 - val_loss: 0.1028
Epoch 3/10
120/120 [==============================] - ETA: 0s - loss: 0.1553

120/120 [==============================] - 45s 379ms/step - loss: 0.1553 - val_loss: 0.0547
Epoch 4/10
120/120 [==============================] - ETA: 0s - loss: 0.1144

120/120 [==============================] - 46s 388ms/step - loss: 0.1144 - val_loss: 0.0514
Epoch 5/10
120/120 [==============================] - ETA: 0s - loss: 0.1121

...

120/120 [==============================] - 46s 386ms/step - loss: 0.0450 - val_loss: 0.0146
Epoch 7/10
120/120 [==============================] - ETA: 0s - loss: 0.0339

120/120 [==============================] - 47s 395ms/step - loss: 0.0339 - val_loss: 0.0270
Epoch 8/10
120/120 [==============================] - ETA: 0s - loss: 0.0373

120/120 [==============================] - 48s 401ms/step - loss: 0.0373 - val_loss: 0.0744
Epoch 9/10
120/120 [==============================] - ETA: 0s - loss: 0.0616

120/120 [==============================] - 46s 384ms/step - loss: 0.0616 - val_loss: 0.0532
Epoch 10/10
120/120 [==============================] - ETA: 0s - loss: 0.0317

120/120 [==============================] - 46s 388ms/step - loss: 0.0317 - val_loss: 0.0577
